In [1]:
# import libraries
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from textblob import TextBlob
import pandas as pd
import numpy as np
import torch
import os

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
df_train = pd.read_csv("data/train.csv")
df_val = pd.read_csv("data/val.csv")
df_test = pd.read_csv("data/test.csv")

In [6]:
df_train.head()

,tokens,tags,sentence,SA
0,['People' 'start' 'their' 'own' 'businesses' '...,[0 0 0 0 0 0 0 0 0],People start their own businesses for many rea...,1
1,['But' 'a' 'chance' 'to' 'fill' 'out' 'sales' ...,[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0],But a chance to fill out sales - tax records i...,1
2,['Red' 'tape' 'is' 'the' 'bugaboo' 'of' 'small...,[0 0 0 0 0 0 0 0 0],Red tape is the bugaboo of small business .,0
3,"['Ironically' ',' 'the' 'person' 'who' 'wants'...",[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,"Ironically , the person who wants to run his o...",0
4,['Yet' 'every' 'business' 'owner' 'has' 'to' '...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,Yet every business owner has to face the mound...,1


In [ ]:
class OntoNoteDataset(Dataset):
    dataset: torch.Tensor

    def __init__(self, df: pd.DataFrame):
        #self.data = df
        pass
         # HAY QUE SABER COMO SON LSO DATOS PARA SABERLO
    def __len__(self):
        return len(self.data)
    
    def __get_item__(self, idx: int):
        row = self.data.iloc[idx]
        row_tensor
        
        return self.data[idx]
    

def load_data()
    





class ElectricDataset(Dataset):

    def __init__(self, dataset: pd.DataFrame, past_days: int) -> None:
        """
        Constructor of ElectricDataset.

        Args:
            dataset: dataset in dataframe format. It has three columns
                (price, feature 1, feature 2) and the index is
                Timedelta format.
            past_days: number of past days to use for the
                prediction.
        """

        # TODO
        self.past_days = past_days  

        daily_prices = dataset["Price"].groupby(dataset.index.date).apply(
            lambda x: x.values[:24]  
        )

        daily_prices = daily_prices.apply(lambda x: np.pad(x, (0, 24 - len(x)), 'constant', constant_values=np.nan))
        price_array = np.array(daily_prices.tolist(), dtype=np.float64)
        self.dataset = torch.tensor(price_array, dtype=torch.float64)

    def __getitem__(self, index: int) -> tuple[torch.Tensor, torch.Tensor]:
       
        past_values: torch.Tensor = self.dataset[index:index+self.past_days]
        current_values: torch.Tensor = self.dataset[index+self.past_days]
        return past_values, current_values


def load_data(
    save_path: str,
    past_days: int = 7,
    batch_size: int = 64,
    shuffle: bool = True,
    drop_last: bool = False,
    num_workers: int = 0,
) -> tuple[DataLoader, DataLoader, DataLoader, float, float]:
    """
    This method returns Dataloaders of the chosen dataset. Use  the
    last 42 weeks of the training dataframe for the validation set.

    Args:
        save_path: path to save the data.
        past_days: number of past days to use for the prediction.
        batch_size: size of batches that wil be created.
        shuffle: indicator of shuffle the data. Defaults to true.
        drop_last: indicator to drop the last batch since it is not
            full. Defaults to False.
        num_workers: num workers for loading the data. Defaults to 0.

    Returns:
        train dataloader.
        val dataloader.
        test dataloader.
        means of price.  # PARA DESNORMALIZAR, OSEA AQUI MORMALIZAMOD LUEGO ENTRENAMOS CON ELLO NORMALIZADO, LUEGO DESNORMALIZO
        stds of price.
    """
    # TODO
    # CADA PALABRA ES 1 DIA QUE ESTA FORMADO POR 24 HORAS

    df_train: pd.DataFrame
    df_test: pd.DataFrame
    df_train, df_test = download_data(save_path)

    price_mean: float = df_train["Price"].mean()
    price_std: float = df_train["Price"].std()

    # Normalizo los datos
    df_train["Price"] = (df_train["Price"] - price_mean) / price_std
    df_test["Price"] = (df_test["Price"] - price_mean) / price_std

    # Cojo los datasets
    total_days: int = len(df_train)
    days_for_training: int = total_days - 42 * 7 * 24
    days_for_validation: int = days_for_training - past_days*24

    train_dataset: Dataset = ElectricDataset(df_train[:days_for_training], past_days)
    val_dataset: Dataset = ElectricDataset(df_train[days_for_validation:], past_days)

    test_dataframe: pd.DataFrame = pd.concat([df_train.iloc[-past_days*24:], df_test])   

    test_dataset: Dataset = ElectricDataset(test_dataframe,past_days)

    train_dataloader: DataLoader = DataLoader(train_dataset,batch_size=batch_size,shuffle=shuffle,num_workers=num_workers, drop_last=drop_last)
    val_dataloader: DataLoader = DataLoader(val_dataset,batch_size=batch_size,shuffle=shuffle,num_workers=num_workers, drop_last=drop_last)
    test_dataloader: DataLoader = DataLoader(test_dataset,batch_size=batch_size,shuffle=shuffle,num_workers=num_workers, drop_last=drop_last)

    return train_dataloader, val_dataloader, test_dataloader, price_mean, price_std


def download_data(path) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Function to download data from for the project.

    Args:
        path: path to save the data
        years_test: year for the test data. Defaults to 2.
        begin_test_date: beginning test date. Defaults to None.
        end_test_date: end test date. Defaults to None.

    Raises:
        IOError: Error when reading dataset with pandas.
        Exception: Starting date for test dataset should be midnight.
        Exception: End date for test dataset should be at 0h or 23h.

    Returns:
        training dataset.
        testing dataset.

    """

    dataset: str = "NP"

    if not os.path.exists(path):
        os.makedirs(path)

    if dataset in ["PJM", "NP", "FR", "BE", "DE"]:
        file_path = os.path.join(path, dataset + ".csv")

        if os.path.exists(file_path):
            data = pd.read_csv(file_path, index_col=0)
        else:
            url_dir = "https://zenodo.org/records/4624805/files/"
            data = pd.read_csv(url_dir + dataset + ".csv", index_col=0)
            data.to_csv(file_path)
    else:
        try:
            file_path = os.path.join(path, dataset + ".csv")
            data = pd.read_csv(file_path, index_col=0)
        except IOError as e:
            raise IOError("%s: %s" % (path, e.strerror))

    data.index = pd.to_datetime(data.index)

    columns = ["Price"]
    n_exogeneous_inputs = len(data.columns) - 1

    for n_ex in range(1, n_exogeneous_inputs + 1):
        columns.append("Exogenous " + str(n_ex))

    data.columns = columns

    # The training and test datasets can be defined by providing a number
    # of years for testing
    # or by providing the init and end date of the test period
    if begin_test_date is None and end_test_date is None:
        number_datapoints = len(data.index)
        number_training_datapoints = number_datapoints - 24 * 364 * years_test

        # We consider that a year is 52 weeks (364 days) instead of the traditional 365
        df_train = data.loc[
            : data.index[0] + pd.Timedelta(hours=number_training_datapoints - 1), :
        ]
        df_test = data.loc[
            data.index[0] + pd.Timedelta(hours=number_training_datapoints) :, :
        ]

    else:
        try:
            begin_test_date = pd.to_datetime(begin_test_date, dayfirst=True)
            end_test_date = pd.to_datetime(end_test_date, dayfirst=True)
        except ValueError:
            print("Provided values for dates are not valid")

        if begin_test_date.hour != 0:
            raise Exception("Starting date for test dataset should be midnight")
        if end_test_date.hour != 23:
            if end_test_date.hour == 0:
                end_test_date = end_test_date + pd.Timedelta(hours=23)
            else:
                raise Exception("End date for test dataset should be at 0h or 23h")

        print("Test datasets: {} - {}".format(begin_test_date, end_test_date))
        df_train = data.loc[: begin_test_date - pd.Timedelta(hours=1), :]
        df_test = data.loc[begin_test_date:end_test_date, :]

    return df_train, df_test


In [2]:
# download dataset
ds = load_dataset("tner/ontonotes5")

In [3]:
entity2tag = {
    "O": 0,
    "B-CARDINAL": 1,
    "B-DATE": 2,
    "I-DATE": 3,
    "B-PERSON": 4,
    "I-PERSON": 5,
    "B-NORP": 6,
    "B-GPE": 7,
    "I-GPE": 8,
    "B-LAW": 9,
    "I-LAW": 10,
    "B-ORG": 11,
    "I-ORG": 12, 
    "B-PERCENT": 13,
    "I-PERCENT": 14, 
    "B-ORDINAL": 15, 
    "B-MONEY": 16, 
    "I-MONEY": 17, 
    "B-WORK_OF_ART": 18, 
    "I-WORK_OF_ART": 19, 
    "B-FAC": 20, 
    "B-TIME": 21, 
    "I-CARDINAL": 22, 
    "B-LOC": 23, 
    "B-QUANTITY": 24, 
    "I-QUANTITY": 25, 
    "I-NORP": 26, 
    "I-LOC": 27, 
    "B-PRODUCT": 28, 
    "I-TIME": 29, 
    "B-EVENT": 30,
    "I-EVENT": 31,
    "I-FAC": 32,
    "B-LANGUAGE": 33,
    "I-PRODUCT": 34,
    "I-ORDINAL": 35,
    "I-LANGUAGE": 36
}


In [4]:
df_train = ds["train"].to_pandas()
df_val = ds["validation"].to_pandas()
df_test = ds["test"].to_pandas()

print(df_train.head())

                                              tokens  \
0  [People, start, their, own, businesses, for, m...   
1  [But, a, chance, to, fill, out, sales, -, tax,...   
2  [Red, tape, is, the, bugaboo, of, small, busin...   
3  [Ironically, ,, the, person, who, wants, to, r...   
4  [Yet, every, business, owner, has, to, face, t...   

                                                tags  
0                        [0, 0, 0, 0, 0, 0, 0, 0, 0]  
1   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]  
2                        [0, 0, 0, 0, 0, 0, 0, 0, 0]  
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  


In [5]:
def analyze_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity < 0:
        return 0  # Negativo
    elif polarity == 0:
        return 1  # Neutro
    else:
        return 2  # Positivo
        
def process_sentences(df):
    # Crear la columna 'sentence' uniendo los tokens
    df["sentence"] = df["tokens"].apply(lambda x: " ".join(x))

    # Aplicar análisis de sentimiento
    df["SA"] = df["sentence"].apply(analyze_sentiment)
    
    return df

# Aplicar a cada dataset
df_train = process_sentences(df_train)
df_val = process_sentences(df_val)
df_test = process_sentences(df_test)

In [6]:
df_train.head()

,tokens,tags,sentence,SA
0,"[People, start, their, own, businesses, for, m...","[0, 0, 0, 0, 0, 0, 0, 0, 0]",People start their own businesses for many rea...,2
1,"[But, a, chance, to, fill, out, sales, -, tax,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",But a chance to fill out sales - tax records i...,2
2,"[Red, tape, is, the, bugaboo, of, small, busin...","[0, 0, 0, 0, 0, 0, 0, 0, 0]",Red tape is the bugaboo of small business .,0
3,"[Ironically, ,, the, person, who, wants, to, r...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Ironically , the person who wants to run his o...",2
4,"[Yet, every, business, owner, has, to, face, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Yet every business owner has to face the mound...,2


In [7]:
df_train.tail(5)

,tokens,tags,sentence,SA
59919,"[Probably, .]","[0, 0]",Probably .,1
59920,"[Why, ?]","[0, 0]",Why ?,1
59921,"[I, do, n't, think, they, can]","[0, 0, 0, 0, 0, 0]",I do n't think they can,1
59922,"[but, I, think, they, 'll, try, .]","[0, 0, 0, 0, 0, 0, 0]",but I think they 'll try .,1
59923,"[You, know, how, people, are, .]","[0, 0, 0, 0, 0, 0]",You know how people are .,1


In [8]:
import spacy
import re

In [9]:
nlp = spacy.load("en_core_web_sm")

CONTRACTIONS = {
    "n't": "not", 
    "'ll": "will", 
    "'re": "are", 
    "'ve": "have", 
    "'m": "am", 
    "'d": "would", 
    "'s": "is", 
    "won't": "will not", 
    "can't": "cannot"
}
IRRELEVANT_WORDS = {"wow", "oops", "ah", "ugh", "yay", "mhm", "`"}

def replace_contractions(text):
    for contraction, replacement in CONTRACTIONS.items():
        text = re.sub(r"\b" + re.escape(contraction) + r"\b", replacement, text)
    return text

def process_tokens(text):
    text = replace_contractions(text)
    text = text.replace("-", "")
    # Crear un objeto Doc de spaCy para cada palabra
    doc = nlp(text)  # Unir las palabras en una cadena y procesarlas con spaCy
    
    # Filtrar tokens: eliminar puntuación, stopwords y lematizar
    processed_words = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.text.lower() not in IRRELEVANT_WORDS]
    
    return processed_words

df_train["tokens"] = df_train["sentence"].apply(process_tokens)

In [10]:
df_train.head(15)

,tokens,tags,sentence,SA
0,"[People, start, business, reason]","[0, 0, 0, 0, 0, 0, 0, 0, 0]",People start their own businesses for many rea...,2
1,"[chance, fill, sale, , tax, record, rarely]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",But a chance to fill out sales - tax records i...,2
2,"[red, tape, bugaboo, small, business]","[0, 0, 0, 0, 0, 0, 0, 0, 0]",Red tape is the bugaboo of small business .,0
3,"[ironically, person, want, run, business, prob...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Ironically , the person who wants to run his o...",2
4,"[business, owner, face, mound, form, regulatio...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Yet every business owner has to face the mound...,2
5,"[hope, change]","[0, 0, 0, 0, 0, 0]",There is hope of change .,1
6,"[week, Sen., Malcolm, Wallop, LRB, R., Wyo, rr...","[2, 3, 0, 0, 4, 5, 0, 6, 0, 7, 8, 0, 0, 0, 0, ...","Last week , Sen. Malcolm Wallop -LRB- R. , Wyo...",0
7,"[great, federal, regulation, mean, large, enti...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",`` A great many federal regulations are meant ...,2
8,"[lawmaker, busy, try, revive, recently, lapse,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 10, 10, 0, 0...",Other lawmakers are busy trying to revive the ...,2
9,"[optimistic, entrepreneur, await, promise, lan...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Thus , optimistic entrepreneurs await a promis...",0


In [11]:
df_train["sentence"].iloc[6]

'Last week , Sen. Malcolm Wallop -LRB- R. , Wyo . -RRB- held hearings on a bill to strengthen an existing law designed to reduce regulatory hassles for small businesses .'

In [12]:
df_val["tokens"] = df_val["sentence"].apply(process_tokens)

In [13]:
df_test["tokens"] = df_test["sentence"].apply(process_tokens)

In [14]:
# save data in .csv
if not os.path.exists("data"):
    os.makedirs("data")

df_train.to_csv(f"data/df_train.csv", index=False)
df_val.to_csv(f"data/df_val.csv", index=False)
df_test.to_csv(f"data/df_test.csv", index=False)